# Proyecto 3 - Aprendizaje no supervisado

- Juan Ignacio Navarro
- Jose David Sánchez
- Steven Badilla

In [1]:
"""
Used libraries for the entire project
"""
import os
import numpy as np
import pandas as pd
from PIL import Image
import random 
import shutil

## Feature engineering

Set de datos llamado Plant_leaf_deseases_dataset_without_augmentation: https://data.mendeley.com/datasets/tywbtsjrjv/1

### Crop and fix Bias in images dataset

In [ ]:
"""
-- Delete unnecessary images --

These background images do not help the model on anything
"""

os.remove("images/original_dataset/Background_without_leaves")

In [45]:
"""
-- Delete images from cropped dataset --

This cell deletes the files in the cropped dataset if any. The
cropped dataset a cropped copy from the additional dataset images.
The images are cropped to show only the most important part of 
the leaves which is the center
"""

cropped_images_folder = 'images/cropped_dataset/'

# get the categories
categories = {}
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    folder_path = os.path.join(cropped_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# remove the files
for category in categories.keys():

    folder_path = os.path.join(cropped_images_folder, category)
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        file_path = os.path.join(cropped_images_folder, category, file_name)
        os.remove(file_path)

In [46]:
"""
-- Crop images borders -- 

This block crops the images borders and saves the new images into the cropped_dataset
"""

# define a different folder to save the cropped images
original_images_folder = 'images/original_dataset/'
analysis_images_folder = 'images/cropped_dataset/'

# define the new size
target_size = (100, 100)

categories = {}

# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(original_images_folder))):
    folder_path = os.path.join(original_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# Create category folders in cropped dataset
for index, folder_name in enumerate(sorted(os.listdir(original_images_folder))):
    folder_path = os.path.join(cropped_images_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)

# crop and save the cropped images
for category in categories.keys():

    cat_files = os.listdir(os.path.join(original_images_folder, category))

    for i, file in enumerate(cat_files):

        # constructing image path
        input_path = os.path.join(original_images_folder, category, file)
        image = Image.open(input_path)

        if image.size[0] < 256 or image.size[1] < 256:
            continue
        
        # get original image size and calculate borders
        width, height = image.size
        left = (width - target_size[0]) // 2
        upper = (height - target_size[1]) // 2
        right = left + target_size[0]
        lower = upper + target_size[1]

        # crop the image
        cropped_image = image.crop((left, upper, right, lower))
        output_path = os.path.join(cropped_images_folder, category, file)
        cropped_image.save(output_path)
        image.close()

In [47]:
"""
-- Check Bias --

Calculate the amount of images to check for possible Bias in the dataset
"""

cropped_images_folder = 'images/cropped_dataset/'
category_amount = []
min_bias = 10e4
min_bias_key = ''
max_bias = 0
max_bias_key = ''

categories = {}
# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    folder_path = os.path.join(cropped_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# get the amount of images for each category
for category in categories.keys():
    folder_path = os.path.join(cropped_images_folder, category)
    image_files = os.listdir(folder_path)
    folder_count = len(image_files)
    if folder_count > max_bias:
        max_bias = folder_count
        max_bias_key = category
    if folder_count < min_bias:
        min_bias = folder_count
        min_bias_key = category
    category_amount.append(folder_count)

# Print the information taken
print("Amount of images in analysis dataset:")
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    print(f"{folder_name:50}: {category_amount[index]:5d}")

print(f"Category with the minimum amount of images: {min_bias_key} with {min_bias}")
print(f"Category with the maximim amount of images: {max_bias_key} with {max_bias}")

Amount of images in analysis dataset:
Apple___Apple_scab                                :   630
Apple___Black_rot                                 :   621
Apple___Cedar_apple_rust                          :   275
Apple___healthy                                   :  1645
Blueberry___healthy                               :  1502
Cherry___Powdery_mildew                           :  1052
Cherry___healthy                                  :   854
Corn___Cercospora_leaf_spot Gray_leaf_spot        :   513
Corn___Common_rust                                :  1192
Corn___Northern_Leaf_Blight                       :   985
Corn___healthy                                    :  1162
Grape___Black_rot                                 :  1180
Grape___Esca_(Black_Measles)                      :  1383
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)        :  1076
Grape___healthy                                   :   423
Orange___Haunglongbing_(Citrus_greening)          :  5507
Peach___Bacterial_spot            

In [43]:
"""
-- Eliminate bias by combining images --

By combining pixels from the images categories it will be possible to add more images to the categories
that need more data
"""

cropped_dataset_folder = "images/cropped_dataset/"

max_bias_folder = f"{cropped_dataset_folder}/{max_bias_key}/"

AMOUNT_IMAGES = 500   

folders = os.listdir(cropped_dataset_folder)

for folder in folders:
    folder_path = os.path.join(cropped_dataset_folder, folder)
    amount_images = len(os.listdir(folder_path))

    if amount_images > 1.5 * min_bias: continue

    random_images = random.sample(os.listdir(folder_path), min_bias)

    counter = 0
    counter_random = 0
    while(counter < AMOUNT_IMAGES):

        for image in os.listdir(folder_path):

            if counter % min_bias == 0: counter_random += 1

            if (counter == AMOUNT_IMAGES): break
            random_image_path = os.path.join(max_bias_folder, random_images[counter_random])
            other_folder_image_path = os.path.join(folder_path, image)

            image1 = Image.open(random_image_path)
            image2 = Image.open(other_folder_image_path)

            if image1.size == (100, 100, 3) or image2.size == (100, 100, 3): continue

            image1.resize((100, 100))
            image2.resize((100, 100))

            array1 = np.array(image1)
            array2 = np.array(image2)

            average_array = (0.5 * array1 + 0.5 * array2).astype(np.uint8)
            combined_image = Image.fromarray(average_array)
            
            combined_image_path = os.path.join(folder_path, f"combined_{counter}.JPG")
            combined_image.save(combined_image_path)
            counter += 1

In [44]:
"""
-- Check Bias --

Calculate the amount of images to check for possible Bias in the dataset
"""
cropped_images_folder = 'images/cropped_dataset/'
category_amount = []
min_bias = 10e4
min_bias_key = ''
max_bias = 0
max_bias_key = ''

categories = {}
# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    folder_path = os.path.join(cropped_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# get the amount of images for each category
for category in categories.keys():
    folder_path = os.path.join(cropped_images_folder, category)
    image_files = os.listdir(folder_path)
    folder_count = len(image_files)
    if folder_count > max_bias:
        max_bias = folder_count
        max_bias_key = category
    if folder_count < min_bias:
        min_bias = folder_count
        min_bias_key = category
    category_amount.append(folder_count)

# Print the information taken
print("Amount of images in analysis dataset:")
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    print(f"{folder_name:50}: {category_amount[index]:5d}")

print(f"Category with the minimum amount of images: {min_bias_key} with {min_bias}")
print(f"Category with the maximim amount of images: {max_bias_key} with {max_bias}")

Amount of images in analysis dataset:
Apple___Apple_scab                                :   630
Apple___Black_rot                                 :   621
Apple___Cedar_apple_rust                          :   775
Apple___healthy                                   :  1645
Blueberry___healthy                               :  1502
Cherry___Powdery_mildew                           :  1052
Cherry___healthy                                  :   854
Corn___Cercospora_leaf_spot Gray_leaf_spot        :   513
Corn___Common_rust                                :  1192
Corn___Northern_Leaf_Blight                       :   985
Corn___healthy                                    :  1162
Grape___Black_rot                                 :  1180
Grape___Esca_(Black_Measles)                      :  1383
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)        :  1076
Grape___healthy                                   :   423
Orange___Haunglongbing_(Citrus_greening)          :  5507
Peach___Bacterial_spot            

In [2]:
"""
-- Delete images from analysis dataset --

This cell deletes the files in the cropped dataset if any. The
cropped dataset a cropped copy from the additional dataset images.
The images are cropped to show only the most important part of 
the leaves which is the center
"""

analysis_images_folder = 'images/analysis_dataset/'

# get the categories
categories = {}
for index, folder_name in enumerate(sorted(os.listdir(analysis_images_folder))):
    folder_path = os.path.join(analysis_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# remove the files
for category in categories.keys():

    folder_path = os.path.join(analysis_images_folder, category)
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        file_path = os.path.join(analysis_images_folder, category, file_name)
        os.remove(file_path)

In [3]:
"""
-- Move images to analysis dataset --

This ignores the extra images in the Bias
"""

# define a different folder to save the cropped images
analysis_images_folder = 'images/analysis_dataset/'
cropped_images_folder = 'images/cropped_dataset/'

categories = {}

# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    folder_path = os.path.join(cropped_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# Create category folders in cropped dataset
for index, folder_name in enumerate(sorted(os.listdir(cropped_images_folder))):
    folder_path = os.path.join(analysis_images_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)


for category in categories.keys():

    cropped_category_folder = os.path.join(cropped_images_folder, category)
    analysis_category_folder = os.path.join(analysis_images_folder, category)

    for i, image in enumerate(os.listdir(cropped_category_folder)[::-1]):

        #if i == min_bias*1.5: break

        cropped_image_path = os.path.join(cropped_category_folder, image)
        analysis_image_path = os.path.join(analysis_category_folder, image)

        shutil.copy(cropped_image_path, analysis_image_path)

In [4]:
"""
-- Check Bias --

Calculate the amount of images to check for possible Bias in the dataset
"""

analysis_images_folder = 'images/analysis_dataset/'
category_amount = []
min_bias = 10e4
min_bias_key = ''
max_bias = 0
max_bias_key = ''

categories = {}
# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(analysis_images_folder))):
    folder_path = os.path.join(analysis_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# get the amount of images for each category
for category in categories.keys():
    folder_path = os.path.join(analysis_images_folder, category)
    image_files = os.listdir(folder_path)
    folder_count = len(image_files)
    if folder_count > max_bias:
        max_bias = folder_count
        max_bias_key = category
    if folder_count < min_bias:
        min_bias = folder_count
        min_bias_key = category
    category_amount.append(folder_count)

print(f"Category with the minimum amount of images: {min_bias_key} with {min_bias}")
print(f"Category with the maximim amount of images: {max_bias_key} with {max_bias}")

Category with the minimum amount of images: Potato___healthy with 152
Category with the maximim amount of images: Orange___Haunglongbing_(Citrus_greening) with 5507


In [5]:
"""
-- Delete contents in resized_dataset folder -- 
"""
resized_images_folder = 'images/resized_dataset/'

# get the categories
categories = {}
for index, folder_name in enumerate(sorted(os.listdir(resized_images_folder))):
    folder_path = os.path.join(resized_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

# remove the files
for category in categories.keys():

    folder_path = os.path.join(resized_images_folder, category)
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        file_path = os.path.join(resized_images_folder, category, file_name)
        os.remove(file_path)

In [ ]:
"""
-- Resize images to 64 x 64 --
"""

analysis_images_folder = 'images/analysis_dataset/'
resized_images_folder = 'images/resized_dataset/'


categories = {}
# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(analysis_images_folder))):
    folder_path = os.path.join(analysis_images_folder, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

for category in categories.keys():

    analysis_category_folder = os.path.join(analysis_images_folder, category)
    resized_category_folder = os.path.join(resized_images_folder, category)

    if not os.path.exists(resized_category_folder):
        os.mkdir(resized_category_folder)

    for image in os.listdir(analysis_category_folder):

        analysis_image_path = os.path.join(analysis_category_folder, image)
        resized_image_path = os.path.join(resized_category_folder, image)

        desired_size = (64, 64)

        image = Image.open(analysis_image_path)

        resized_image = image.resize(desired_size)

        resized_image.save(resized_image_path)


### Load dataset

## Autoencoder

## Experimento 1

## Experimento 2